In [1]:
#install requirements
!pip3 install --upgrade datasets transformers cohere langchain qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 6.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.5/379.5 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 67.1 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 14.1 MB/s eta 0:00:00
     ━

In [2]:
# For a bigger dataset we can ingest the given Contract dataset which is about 8GB in size 
# For a hackathon prototype we have only ingested subset of CUAD dataset
# !wget contracts.tar.gz 'https://drive.google.com/uc?export=download&id=1of37X0hAhECQ3BN_004D8gm6V88tgZaB&confirm=t'
# !tar -zxvf contracts.tar.gz --directory data

In [4]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("cuad")

Generating train split:   0%|          | 0/22450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4182 [00:00<?, ? examples/s]

Dataset cuad downloaded and prepared to /root/.cache/huggingface/datasets/cuad/default/1.0.0/01ed7dc61ab84230462731422e77cbb6f54ea8590b22a2d881b594f4d7f3746c. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
#since we only require the title and context data we can merge train and test to use complete set
df = dataset['train'].to_pandas()
df_test = dataset['test'].to_pandas()
df = pd.concat([df,df_test],ignore_index=True)

In [8]:
df.columns

Index(['id', 'title', 'context', 'question', 'answers'], dtype='object')

In [12]:
import re

#Remove punctuations only from title and remove whitespace and newline chars from both context and title
def preprocess(text,title):
    if title:
        text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = text.lower()
    return text

In [15]:
df.isnull().sum()

id          0
title       0
context     0
question    0
answers     0
dtype: int64

In [16]:
len(df)

26632

In [17]:
#CUAD Dataset has same documents with different QnA pairs, we need only unique documents
df.drop_duplicates(subset='context',inplace=True)

In [18]:
len(df)

509

In [19]:
#preprocessing
df['title_processed'] = df.apply(lambda x: preprocess(x['title'],True),axis=1)
df['context_processed'] = df.apply(lambda x: preprocess(x['context'],False),axis=1)

In [20]:
df['title'][0]

'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT'

In [21]:
df

,id,title,context,question,answers,title_processed,context_processed
0,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,EXHIBIT 10.6\n\n ...,Highlight the parts (if any) of this contract ...,"{'text': ['DISTRIBUTOR AGREEMENT'], 'answer_st...",limeenergyco_09_09_1999 ex 10 distributor agre...,exhibit 10.6 distributor agreement this distri...
68,"WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION A...","WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION A...",Exhibit 10.26 CONFIDENTIAL TREATMENT HAS BE...,Highlight the parts (if any) of this contract ...,{'text': ['Promotion and Distribution Agreemen...,whitesmoke inc_11_08_2011 ex 10 26 promotion a...,exhibit 10.26 confidential treatment has been ...
124,NELNETINC_04_08_2020-EX-1-JOINT FILING AGREEME...,NELNETINC_04_08_2020-EX-1-JOINT FILING AGREEMENT,Exhibit 1\n\nJOINT FILING AGREEMENT\n\nThe und...,Highlight the parts (if any) of this contract ...,"{'text': ['JOINT FILING AGREEMENT'], 'answer_s...",nelnetinc_04_08_2020 ex 1 joint filing agreement,exhibit 1 joint filing agreement the undersign...
166,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...,REDACTED COPY\n\nCONFIDENTIAL TREATMENT REQUES...,Highlight the parts (if any) of this contract ...,"{'text': ['ENDORSEMENT AGREEMENT'], 'answer_st...",adamsgolfinc_03_21_2005 ex 10 17 endorsement a...,redacted copy confidential treatment requested...
217,"KIROMICBIOPHARMA,INC_05_11_2020-EX-10.23-CONSU...","KIROMICBIOPHARMA,INC_05_11_2020-EX-10.23-CONSU...",Exhibit 10.23 Corporate Address Fannin South P...,Highlight the parts (if any) of this contract ...,"{'text': ['CONSULTING AGREEMENT'], 'answer_sta...",kiromicbiopharma inc_05_11_2020 ex 10 23 consu...,exhibit 10.23 corporate address fannin south p...
...,...,...,...,...,...,...,...
26427,ALCOSTORESINC_12_14_2005-EX-10.26-AGENCY AGREE...,ALCOSTORESINC_12_14_2005-EX-10.26-AGENCY AGREE...,Exhibit 10.26\n\nAGENCY AGREEMENT\n\nTHIS AGEN...,Highlight the parts (if any) of this contract ...,"{'text': ['AGENCY AGREEMENT'], 'answer_start':...",alcostoresinc_12_14_2005 ex 10 26 agency agree...,exhibit 10.26 agency agreement this agency agr...
26468,INGEVITYCORP_05_16_2016-EX-10.5-INTELLECTUAL P...,INGEVITYCORP_05_16_2016-EX-10.5-INTELLECTUAL P...,Exhibit 10.5 INTELLECTUAL PROPERTY AGREEMENT...,Highlight the parts (if any) of this contract ...,"{'text': ['INTELLECTUAL PROPERTY AGREEMENT'], ...",ingevitycorp_05_16_2016 ex 10 5 intellectual p...,exhibit 10.5 intellectual property agreement t...
26509,"NOVOINTEGRATEDSCIENCES,INC_12_23_2019-EX-10.1-...","NOVOINTEGRATEDSCIENCES,INC_12_23_2019-EX-10.1-...",Exhibit 10.1 JOINT VENTURE AGREEMENT BETWEEN N...,Highlight the parts (if any) of this contract ...,"{'text': ['JOINT VENTURE AGREEMENT'], 'answer_...",novointegratedsciences inc_12_23_2019 ex 10 1 ...,exhibit 10.1 joint venture agreement between n...
26550,XinhuaSportsEntertainmentLtd_20070221_F-1_EX-9...,XinhuaSportsEntertainmentLtd_20070221_F-1_EX-9...,EXHIBIT 99.4\n\n DA...,Highlight the parts (if any) of this contract ...,"{'text': ['CONTENT LICENSE AGREEMENT'], 'answe...",xinhuasportsentertainmentltd_20070221_f 1_ex 9...,exhibit 99.4 dated 15th december 2001 china ec...


In [22]:
import cohere
# co = cohere.Client('ZGZyUAQN28WdDqOtiO6WaFRv7jMts41VNsMEEJhx')
def callable_embed(sentences):
    co = cohere.Client('enter_api_key')
    response = co.embed(
        model='multilingual-22-12',
        texts=sentences)
    return response.embeddings

In [25]:
# we are using free tier of cohere api which lets us use 100 calls to embedding api per minute hence we need
# following functions to overcome ratelimit
from itertools import islice
import aiohttp
import asyncio
import json
from tqdm.notebook import tqdm
tqdm.pandas()
import time

def chunked_iterable(iterable, chunk_size):
    it = iter(iterable)
    while True:
        chunk = list(islice(it, chunk_size))
        if not chunk:
            break
        yield chunk
        
def get_collection_embeddings(str_list):
    embeddings = [] 
    max_sentences = 95
    
    for str_list_chunk in tqdm(chunked_iterable(str_list, max_sentences)):
        results = callable_embed(str_list_chunk)
        time.sleep(61)
        embeddings.extend(results)
    
    return embeddings

In [27]:
# we need to count number of tokens to send to embedding api as multilingual model truncates sequence length 
# greater than 512
def wordcount(text):
    x = text.split(' ')
    return len(x)

In [28]:
df['context_len'] = df['context_processed'].apply(wordcount)

In [29]:
# separating out >512 seq to explode into smaller chunks for embedding
# this will match partial documents later in neural search where we can use the unique ones to create context for QA
df_process = df[df['context_len']>512]
df = df[df['context_len']<512]

In [30]:
df.reset_index(inplace=True,drop=True)
df_process.reset_index(inplace=True,drop=True)

In [31]:
len(df_process)

482

In [32]:
# If seq len > 512 explode into multiple observations for context_processed
def explode_text(df, col_name, max_words):
    new_rows = []
    for index, row in df.iterrows():
        words = row[col_name].split()
        if len(words) > max_words:
            for i in range(0, len(words), max_words):
                new_row = row.copy()
                new_row[col_name] = ' '.join(words[i:i+max_words])
                new_rows.append(new_row)
        else:
            new_rows.append(row)
    return pd.DataFrame(new_rows)

In [33]:
df_exploded = explode_text(df_process, 'context_processed', 512)

In [34]:
df_exploded.reset_index(inplace=True,drop=True)

In [35]:
# merge the processed docs
df = pd.concat([df,df_exploded])

In [36]:
df.reset_index(inplace=True,drop=True)

In [37]:
df['context_len'] = df['context_processed'].apply(wordcount)
df['title_len'] = df['title_processed'].apply(wordcount)

In [38]:
df[df['title_len']>512]

,id,title,context,question,answers,title_processed,context_processed,context_len,title_len


In [38]:
df[df['context_len']>512]

,id,title,context,question,answers,title_processed,context_processed,context_len,title_len


In [39]:
#call cohere multilingual model embedding api to create embeddings
embeddings_title = get_collection_attributes(list(df['title_processed']))

0it [00:00, ?it/s]

In [40]:
#call cohere multilingual model embedding api to create embeddings
embeddings_context = get_collection_attributes(list(df['context_processed']))

0it [00:00, ?it/s]

In [47]:
len(list(df['context_processed']))

8085

In [49]:
len(embeddings_title[0]) #embedding size

768

In [41]:
df['embeddings_title'] = embeddings_title
df['embeddings_context'] = embeddings_context

In [42]:
# df.to_pickle('mlk.pkl')

## Initiate Qdrant Client

In [171]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

client = QdrantClient("enter_qdrant_host_url", api_key="enter_api_key")

# client.recreate_collection(
#     collection_name="cuad_semantic",
#     vectors_config={
#         "title_embeddings": models.VectorParams(size=768, distance=models.Distance.COSINE),
#         "context_embeddings": models.VectorParams(size=768, distance=models.Distance.COSINE),
#     }
# )

client.recreate_collection(
    collection_name="cuad_semantic_title",
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE)
)

client.recreate_collection(
    collection_name="cuad_semantic_context",
    vectors_config= models.VectorParams(size=768, distance=models.Distance.COSINE)
)

True

### We will create 2 separate indexes for title and context which can be later used for selective neural search

In [172]:
df_payload = df[['id','title','context']]

In [173]:
embeddings_title_f = [
    # Conversion to float is required for Qdrant
    list(map(float, vector)) 
    for vector in embeddings_title
]

embeddings_context_f = [
    # Conversion to float is required for Qdrant
    list(map(float, vector)) 
    for vector in embeddings_context
]

In [174]:
df_payload.reset_index(inplace=True)

In [175]:
ids = list(df_payload['index'])

In [176]:
dict_records = df_payload.to_dict(orient='records')

In [177]:
dict_records[3]

{'index': 3,
 'id': 'BANGIINC_05_25_2005-EX-10-Premium Managed Hosting Agreement__Document Name_0',
 'title': 'BANGIINC_05_25_2005-EX-10-Premium Managed Hosting Agreement',
 'context': '03/01/05\n\n607-1295 Richards Street   604.684.2255\n\nVancouver, BC V6B1B7   deep@rrt.ca   deep systems\n\n\n\n\n\nPremium Managed Hosting Agreement\n\nThis is a managed hosting agreement between AstroNutrition.com and deep systems.  The effective term is 12 months beginning March 1, 2005 and ending February 28, 2006.\n\nIncluded Monthly Services\n\nSystem Administration\n\nManagement of SMTP, IMAP, DNS, SQL database and HTTP server software and systems for the AstroNutrition.com domain and web site.  This includes regular off-site backups of the website itself and the database.\n\nChange Management\n\nManagement of site source code and integration of contributed software updates and bug fixes into zencart is included.\n\nPersonal Technical Support\n\n24 x 7 emergency phone support and 1-business-day e

In [ ]:
# qdrant_client.upload_collection(
#     collection_name='cuad_semantic',
#     vectors=vectors,
#     payload=payload,
#     ids=None,  # Vector ids will be assigned automatically
#     batch_size=256  # How many vectors will be uploaded in a single request?
# )

In [178]:
# client.delete_collection(collection_name="cuad_semantic")

In [179]:
def upsert_qdrant(idx,collection_name,embed,payload):
    try:
        payload_df = pd.DataFrame(columns=['id', 'title', 'context'])
        payload_df.loc[0, :] = payload.values
        payload = payload_df.to_dict(orient='records')
        payload = payload[0]
        res = client.upsert(
        collection_name= collection_name,
        points=[
            models.PointStruct(
                    id=idx,
                    payload=payload,
                    vector=embed
                )
            ]
        )
        if str(res.status) != 'UpdateStatus.COMPLETED':
            print(res.status)
            print("retrying")
            res_ret = client.upsert(
            collection_name=collection_name,
            points=[
                models.PointStruct(
                        id=idx,
                        payload=payload,
                        vector=embed
                    )
                ]
            )
            if str(res_ret.status) != 'UpdateStatus.COMPLETED':
                print(f"retry failed: {res_ret.status}")
            return res_ret
        return res.status
    except Exception as e:
        print(e,idx)
        return -1

In [112]:
df.reset_index(inplace=True)

In [116]:
df.rename({'index':'id_num'},axis=1,inplace=True)

In [180]:
df['embeddings_title'] = embeddings_title_f
df['embeddings_context'] = embeddings_context_f

In [181]:
df.columns

Index(['id_num', 'id', 'title', 'context', 'question', 'answers',
       'title_processed', 'context_processed', 'context_len', 'title_len',
       'embeddings_title', 'embeddings_context', 'resp'],
      dtype='object')

In [121]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
#index title to qdrant
df['resp_title'] = df.progress_apply(lambda x: upsert_qdrant(x['id_num'],"cuad_semantic_title",x['embeddings_title'],x[['id','title','context']]),axis=1)


  0%|          | 0/8085 [00:00<?, ?it/s]

Unexpected Response: 502 (Bad Gateway)
Raw response content:
b'Bad Gateway' 167


In [ ]:
# index context to qdrant
df['resp_context'] = df.progress_apply(lambda x: upsert_qdrant(x['id_num'],"cuad_semantic_context",x['embeddings_context'],x['embeddings_context'],x[['id','title','context']]),axis=1)


In [160]:
df['resp'].value_counts()

UpdateStatus.COMPLETED    8085
Name: resp, dtype: int64

In [162]:
client.get_collection(collection_name="cuad_semantic_context")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=16170, indexed_vectors_count=0, points_count=8085, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors={'context_embeddings': VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>), 'title_embeddings': VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>)}, shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0)), payload_schema={})

In [36]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import cohere
class NeuralSearcher:

    def __init__(self, collection_name):
        self.collection_name = collection_name
        # initialize Qdrant client
        self.qdrant_client = QdrantClient("enter_qdrant_host_url", api_key="enter_api_key")

    def callable_embed(self,sentences):
      co = cohere.Client('enter_api_key')
      response = co.embed(
        model='multilingual-22-12',
        texts=[sentences])
      r = response.embeddings
      embeddings_title_f = [
        # Conversion to float is required for Qdrant
        list(map(float, vector)) 
        for vector in r
    ]
      return embeddings_title_f[0]
        
    
    def search(self, text: str):
        # Convert text query into vector
        vector = self.callable_embed(text)
        # print(vector)
        # Use `vector` for search for closest vectors in the collection
        search_result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            query_filter=None,
            with_payload=True,# We don't want any filters for now
            limit=50  # 5 the most closest results is enough
        )
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function we are interested in payload only
        payloads = [hit.payload for hit in search_result]
        return payloads

In [ ]:
ns = NeuralSearcher('cuad_semantic_title')
pl = ns.search('bangi inc managed hosting agreement')

In [38]:
len(pl)

50

In [39]:
for i in pl:
    print(i['title'])

BANGIINC_05_25_2005-EX-10-Premium Managed Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_EX-10.2_Hosting Agreement
VitalibisInc_20180316_8-K_EX-10.2_11100168_

## Using Langchain to call cohere's generate api which will provide natural language output

In [25]:
from langchain import VectorDBQA
from langchain.embeddings import CohereEmbeddings

In [26]:
import os
os.environ["COHERE_API_KEY"] = "enter_api_key"

In [ ]:
from langchain.embeddings import CohereEmbeddings
cohere_embed = CohereEmbeddings(cohere_api_key="enter_api_key")
from langchain.llms import Cohere
cohere_model = Cohere(cohere_api_key="enter_api_key")
from langchain.vectorstores import Qdrant
from langchain import VectorDBQA, OpenAI


doc_store = Qdrant.from_texts(
   answers_list, cohere_embed, host="265561d1-4edd-410c-8810-8cde70574155.us-east-1-0.aws.cloud.qdrant.io",
   api_key="3_zBqdsJ0M50ZUnBOmPB_3BKqiJKk6_cFGNsNi7BWVrD-B3o8sdbgQ"
)



# At this stage all the possible answers are already stored in Qdrant, so we can define the whole QA chain.


llm = cohere_model
qa = VectorDBQA.from_chain_type(
   llm=cohere_model,
   chain_type="refine",
   vectorstore=doc_store,
   return_source_documents=False,
)




import pandas as pd


search_res = pd.DataFrame(pl)


search_res.drop_duplicates(['id'], inplace=True)


search_res.shape


answers_list = search_res['context'].to_list()




question = "tell me about reganholdingcorp in March, 2008"
